# Kurs 7

## Metaprogramming

Der Begriff Metaprogramming heißt soviel wie: Code, der Code generiert. Eine der schönen Eigenschaften von Julia ist, dass Metaprogramming hier sehr elegant funktioniert. In Sprachen wie C++ ist das immer absolut nervig, weil man sozusagen einen extra Überbau über die Sprache benötigt, der den Code manipuliert, *bevor* er kompiliert wird. Insbesondere also *bevor* der kompilierte Code ausgeführt werden kann. Das sind dann sogenannte preprocessor directives.

In Julia verwendet man stattdessen sogenannte Makros. Der große Vorteil gegenüber zu etwa C++ ist, dass man hier durch den JIT beispielsweise einen Variablennahmen mit regex *zur Laufzeit* erzeugen und dann im nächsten Schritt ein Makro zur Generierung von neuem Code mit dem Variablennahmen verwenden kann. Das heißt im Klartext: Diese strikte Trennung von Codeerzeugung und Codeausführung gibt es nicht mehr.

Offtopic erklärt das ganz gut den (mehr oder weniger lustigen) Witz vom Julia-Mitentwickler Jeff Bezanson, der Name Julia könne auch für "Jeff's uncommon lisp is automated" stehen. Lisp ist nämlich eine Programmiersprache deren Stärke insbesondere das Metaprogramming ist. Julias Funktionalität in diesem Bereich kommt also nicht von ungefähr.

### Expressions

Um diese kleine Vorbemerkung ein bisschen plastischer zu machen, begeben wir uns nun auf eine Mini-Reise durch die Ausführungschritte von Julia-Code (ausführlichere Infos [hier](https://docs.julialang.org/en/v1/devdocs/eval/)). Wenn wir in Julia eine Zeile Code eingeben, dann haben wir zunächst ja einfach nur Text, also einen String. Dieser wird dann in eine `Expr` (expression) umgewandelt. Das nennt man `parsen`:

In [11]:
input_expr = Meta.parse("1 + 2")

:(1 + 2)

In [13]:
input_expr |> typeof

Expr

Unsere expression wird schlussendlich in ausführbaren Maschinencode übersetzt und ausgeführt:

In [34]:
eval(input_expr) # Nebenbemerkung: eval läuft IMMER in der global scope!

3

Wenn wir jetzt aber nochmal einen Schritt zurück machen, dann sehen wir, dass in der `Expr` unser Eingabetext in eine hierarchische Baumstruktur gebracht wurde:

In [16]:
input_expr |> dump

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 2


In [21]:
input_expr.head

:call

In [22]:
input_expr.args

3-element Vector{Any}:
  :+
 1
 2

Dabei gibt es immer einen *head* und einen *body*, also sozusagen Stamm und Äste. In unserem Fall ist der head `:call`, weil wir den Operator `+` auf die Inputs `1` und `2` anwenden. Betrachten wir ein etwas schwierigeres Beispiel:

In [19]:
advanced_expr = :(f(1) + 2) # wir erstellen uns direkt eine Expr

:(f(1) + 2)

Hier stellen wir nun fest, dass anstelle der `1` eine weitere (geschachtelte) Expression getreten ist:

In [20]:
advanced_expr |> dump

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol f
        2: Int64 1
    3: Int64 2


Ganz analog ist in dieser der head `:call`, weil wir wir einen Funktionsaufruf (call) `f(x)` haben.

In [24]:
advanced_expr.args[2].head

:call

Außerdem können wir (ähnlich wie bei Strings) *interpolation* nutzen:

In [35]:
x = 2
:(sqrt($x))

:(sqrt(2))

### Mickey Mouse Example

Wofür brauchen wir nun Makros? Nehmen wir einmal an, wir würden gerne die Variablen ```a``` bis ```z``` so belegen, dass ```a = 1, ..., z = 26```. Dann wollen wir das natürlich nicht händisch ausschreiben, sondern möglichst arbeitssparend tun. Dafür können wir das Makro `@eval` nutzen:

In [4]:
# Anfangsbuchstabe (character) ist a
ch = 'a'

for i in 1:26
    # wir nutzen nun das Makro, weil wir so nicht extra eine Expr erzeugen müssen
    @eval $(Symbol(ch)) = $i # Interpolation mit $
    ch += 1
end

y

25

Jetzt kann man sich fragen: Warum genau sollte man das tun wollen? Variablen derart zu benennen ist wahrscheinlich Quatsch, stattdessen würde man wohl eher mit dem Array `collect(1:26)` arbeiten. Eine sinnvolle Anwendung findet man aber zum Beispiel beim Makro `@assert`:

In [1]:
@macroexpand @assert (a == 1) "a ist nicht 1!!"

:(if a == 1
      nothing
  else
      Base.throw(Base.AssertionError("a ist nicht 1!!"))
  end)

### Eigene Makros

Grundsätzlich gibt es viele Zwecke für Metaprogramming, aber in der Regel fällt die Nutzung von Makros in eine von zwei Kategorien:

- Wir wollen neue "language features" implementieren
- Wir wollen syntactic sugar (hübschere Schreibweisen)

Betrachten wir ein einfaches Makro, das quadriert:

In [1]:
macro squared(ex)
    return :($(ex) * $(ex))
end

@squared(2)

4

In [5]:
@macroexpand @squared(2)

:(2 * 2)

Auf den ersten Blick sieht alles gut aus, allerdings gibt es ein Problem:

In [2]:
function foo()
    x = 2
    return @squared x
end

foo (generic function with 1 method)

In [3]:
foo()

UndefVarError: UndefVarError: x not defined

Unser Makro interpretiert `x` als eine globale Variable und nicht als lokale Variable in der Funktion! Der Versuch, dort den entsprechenden Wert reinzukopieren, schlägt also fehl.

In [40]:
@code_lowered foo()

CodeInfo(
1 ─      x = 2
│   %2 = Main.x * Main.x
└──      return %2
)

Dieses Problem lässt sich durch *escaping* beheben. Wir teilen unserem Makro mit, dass `ex` vom Compiler in Ruhe gelassen werden sollte.

In [41]:
macro squared(ex)
    return :($(esc(ex)) * $(esc(ex)))
end

function foo()
    x = 2
    return @squared x
end

foo (generic function with 1 method)

In [42]:
foo()

4

Hier ist ein weiteres Beispiel, wo escaping notwendig ist; der Compiler ersetzt Variablen- und Funktionsnamen stets durch eigene. Das wollen wir hier nicht.

In [8]:
macro trick(expr)
    trick_msg = :(println("Wir benutzen eine trickreiche Funktion!"))
    return Expr(:block, trick_msg, expr)
end

@trick (macro with 1 method)

In [15]:
@trick bar(x) = 1

Wir benutzen eine trickreiche Funktion!


bar (generic function with 1 method)

In [10]:
macro trick(expr)
    trick_msg = :(println("Wir benutzen eine trickreiche Funktion!"))
    return esc(Expr(:block, trick_msg, expr))
end

@trick (macro with 1 method)

In [16]:
@trick foobar(x) = 1

Wir benutzen eine trickreiche Funktion!


foobar (generic function with 1 method)

Wichtige Makros sind außerdem
- `@time`
- `@macroexpand`
- `@which`
- `@edit`

### Ausblick: Loop unrolling (SIMD)

Ein anwendungsnäheres Beispiel als oben wäre etwa das sogenannte loop unrolling. Sagen wir, wir haben einen Vektor, der als Länge ein Vielfaches von 4 hat. Nun muss man wissen: Viele Hardwarearchitekturen haben beschleunigte Operationen implementiert, wenn man viele punktweise (gleiche) Operationen durchführt. Das nennt man auch SIMD (single instruction, multiple data).
Weil Loops erst zur Laufzeit ausgeführt werden, weiß der Kompiler aber nicht immer, dass wir hier so etwas vorliegen haben. Wir können uns also ein Makro schreiben, das aus
```
for i=1:4n
    c[i] = a[i]*b[i]
end
```
den Code
```
for i=1:n
    c[i*4]     = a[i*4]    *b[i*4]
    c[i*4 + 1] = a[i*4 + 1]*b[i*4 + 1]
    c[i*4 + 2] = a[i*4 + 2]*b[i*4 + 2]
    c[i*4 + 3] = a[i*4 + 3]*b[i*4 + 3]
end
```
generiert.

Auf diese Weise weiß der Compiler, dass hier sozusagen 4 mal die gleiche Operation durchgeführt wird und kann diesen Prozess optimieren.

### Ausblick: Eigene Syntax bzw. Modelldefinitionen

Ebenfalls praktisch sind Makros beim Erstellen von eigener Modellsyntax. Ein Beispiel aus eigener Hand wäre etwa die Definition eines makroökonomischen Modells. Um jetzt nicht zu sehr abzuschweifen, hier die Intuition nur ganz kurz: Man bekommt als Problem ein (nichtlineares) Gleichungssystem und will da einen Solver draufwerfen. Dazu kann man das Gleichungssystem in die Form $f(x) = 0$ bringen, wobei $x = (x_1, ..., x_n)$ ein Array in Julia ist.

Realiter hat man aber viele verschiedene Modelle mit jeweils 10-100 Gleichungen und will daher das Modell nicht von Hand umstellen. Das geht aber mit Metaprogramming automatisch! In dem untenstehenden Screenshot sieht man das eigene Makro ```@sfc_model``` mit verschiedenen Submakros, womit man das Modell definieren kann. Durch ein ```Dict``` werden Variablennamen automatisch auf Arrayindizes gemapt.

![label](../graphics/SFC_1.png) 

Hier sieht man den Julia-Ouput; es wird (magic!) automatisch eine Funktion ```f!``` erzeugt, die (wenn die exogenen Variablen bzw. Parameter bereitgestellt werden) man dem Solver übergeben kann.

![Alt text](../graphics/SFC_2.png)

Die Lösung könnte mithilfe von [NLsolve.jl](https://github.com/JuliaNLSolvers/NLsolve.jl), wobei ```F``` Ergebnis und ```x``` Input der Funktion ist, dann so aussehen:
```
nlsolve((F, x) -> f!(F, x, lags, exos, params), initial_values, autodiff = :forward)
```